# Content

## 1. EDA (Exploratory Data Analysis) and data processing

   ###  1.1 Dealing with null values


## 2. Appylication of some simple and basic models 


   ### 2.1 First Classification method applied: SVC
   
   ### 2.2  Second Classification method applied: DecisionTree
   
   ### 2.3. Conclusion from naive application of basic tree and linear models on raw data, with effectless data   engineering conducted. 
   
   
## 3. Dive into data engineering and data processing.


   ### 3.1 Some observations before data engineering
   
   ### 3.2 Getting artists' nationalities using a package called Name2nat
   
   ### 3.3 Third classification method applied: RandomForest
   
   ### 3.4 Fourth classification method: Xgboost
   
   
## 4. More Data Engineering

   ### 4.1 Examine the incompleteness of our data
   
   ### 4.2 Filling nationality and sexuality information
   
   ### 4.3 More detailed information to complement
   
   ### 4.4 Trying our data on RandomForest and Xgboost
   
   ### 4.5 Generating new features from existing features.

   ### 4.6 Sentiment analysis on title
   
## 5. Some conclusions 

In [1]:
# Basic libraries necessary for data processing  and performance measuring
import pandas as pd
import warnings
warnings.simplefilter('ignore')
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline



# Classification models
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier


# Additional libraries for resampling
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTEENN

# Libraries for NLP on text precessing 
from name2nat import Name2nat
import stanza

# Feature engineering tools
import featuretools as ft

# 1. Exploratory Data Analysis

In [2]:
# Loading training and test datasets into dataframes
Train_data = pd.read_csv(r'D:\Jupyter Notebooks\CS985 Assignment1\CS98XClassificationTrain.csv')
Test_data = pd.read_csv(r'D:\Jupyter Notebooks\CS985 Assignment1\CS98XClassificationTest.csv')

# Inspecting the first 5 rows of the training dataset
Train_data.head()

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top genre
0,1,My Happiness,Connie Francis,1996,107,31,45,-8,13,28,150,75,3,44,adult standards
1,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,47,139,49,3,37,NaN
2,3,How Deep Is Your Love,Bee Gees,1979,105,36,63,-9,13,67,245,11,3,77,adult standards
3,4,Woman in Love,Barbra Streisand,1980,170,28,47,-16,13,33,232,25,3,67,adult standards
4,5,Goodbye Yellow Brick Road - Remastered 2014,Elton John,1973,121,47,56,-8,15,40,193,45,3,63,glam rock


In [3]:
# For convenience of showing result, we sneakily grab the test label from Regression problem.
Test_data_label = pd.read_csv(r'D:\Jupyter Notebooks\CS985 Assignment1\CS98XRegressionTest.csv')
Test_data_label = Test_data_label.drop(labels = 66,axis = 0)

In [4]:
# Inspecting the shape of the training dataset
print(f"The training data has {Train_data.shape[0]} instances and {Train_data.shape[1]} features.")

The training data has 453 instances and 15 features.


In [5]:
# Total number of instances
Sum = Train_data.shape[0]+Test_data.shape[0]
print('Total instances:%d'%Sum)

# Inspecting the test-training ratio
print(f"The ratio of test to train is {100*Test_data.shape[0]/Sum:.2f}:{100*Train_data.shape[0]/Sum:.2f}.")

Total instances:566
The ratio of test to train is 19.96:80.04.


In [6]:
# Inspecting the features in the training dataset
# We can see that 3 columns are categorical features and the other 12 are numerical
Train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453 entries, 0 to 452
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         453 non-null    int64 
 1   title      453 non-null    object
 2   artist     453 non-null    object
 3   year       453 non-null    int64 
 4   bpm        453 non-null    int64 
 5   nrgy       453 non-null    int64 
 6   dnce       453 non-null    int64 
 7   dB         453 non-null    int64 
 8   live       453 non-null    int64 
 9   val        453 non-null    int64 
 10  dur        453 non-null    int64 
 11  acous      453 non-null    int64 
 12  spch       453 non-null    int64 
 13  pop        453 non-null    int64 
 14  top genre  438 non-null    object
dtypes: int64(12), object(3)
memory usage: 53.2+ KB


In [7]:
# No missing value in test dataset.
Test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      113 non-null    int64 
 1   title   113 non-null    object
 2   artist  113 non-null    object
 3   year    113 non-null    int64 
 4   bpm     113 non-null    int64 
 5   nrgy    113 non-null    int64 
 6   dnce    113 non-null    int64 
 7   dB      113 non-null    int64 
 8   live    113 non-null    int64 
 9   val     113 non-null    int64 
 10  dur     113 non-null    int64 
 11  acous   113 non-null    int64 
 12  spch    113 non-null    int64 
 13  pop     113 non-null    int64 
dtypes: int64(12), object(2)
memory usage: 12.5+ KB


In [8]:
# Check if there is any missing value.
# 15 values are missing in 'top genre'. That is 15/453 = 3.42% of all the genres. Let's see if we can save it by adding to 
# them some values manually.
Train_data.isnull().sum()

Id            0
title         0
artist        0
year          0
bpm           0
nrgy          0
dnce          0
dB            0
live          0
val           0
dur           0
acous         0
spch          0
pop           0
top genre    15
dtype: int64

## 1.1 Dealing with null values

### After Googling the information of these songs, we found that most of their genres are very ambiguous and most belong to pop song, which is a rather minor genre in our label to be classified. If we filling in 'pop' for all of them, we might add biases to the dataset and thus lower our accuracy of prediction because the genres of these song on Google may not be very accurate. So we decide to fill in genres for them by comparing their numerical features with the top 3 most frequent genres in the training set: 'adult standards', 'dance pop' and 'glam rock'. However one of them could not fit well into any of the 3 genres so we decide to drop it later on. 


In [9]:
# Only 5 pop songs in the original training dataset so we decide not to add those songs of missing genre to it although
# they appear to be pop songs.
Train_data[Train_data['top genre'] == 'pop']

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top genre
5,6,Grenade,Bruno Mars,2010,110,56,71,-7,12,23,223,15,6,74,pop
49,50,Fireflies,Owl City,2009,180,66,51,-7,12,46,228,3,4,78,pop
85,86,Heal the World,Michael Jackson,1991,81,49,53,-8,10,10,384,54,5,67,pop
364,365,Just the Way You Are,Bruno Mars,2010,109,84,64,-5,9,43,221,2,4,77,pop
386,387,Rock with You - Single Version,Michael Jackson,1979,114,54,81,-13,16,85,221,18,4,72,pop


In [10]:
# Could not decide which genre to fill in as its numerical features don't match well with any of them. 
Train_data[Train_data['Id'] == 27]

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top genre
26,27,Drinks On The House,Green Martini Keys,2016,97,16,57,-16,11,15,181,100,5,28,NaN


In [11]:
# Fill in genres
Train_data['top genre'][1] = 'adult standards'
Train_data['top genre'][220] = 'adult standards'
Train_data['top genre'][8] = 'adult standards'
Train_data['top genre'][72] = 'adult standards'   
Train_data['top genre'][90] = 'adult standards'
Train_data['top genre'][98] = 'adult standards'   
Train_data['top genre'][252] = 'adult standards'
Train_data['top genre'][265] = 'adult standards'
Train_data['top genre'][266] = 'adult standards'
Train_data['top genre'][314] = 'adult standards'
Train_data['top genre'][330] = 'adult standards'
Train_data['top genre'][362] = 'dance pop'
Train_data['top genre'][380] = 'adult standards'
Train_data['top genre'][314] = 'adult standards'
Train_data['top genre'][443] = 'album rock'

In [12]:
Train_data = Train_data.dropna()

In [13]:
# For the purpose of classification, we look at how many genres there are in the training data. The number is 86.
Train_data['top genre'].value_counts()

adult standards       80
album rock            67
dance pop             62
brill building pop    16
glam rock             16
                      ..
bow pop                1
australian rock        1
boogaloo               1
british comedy         1
alternative rock       1
Name: top genre, Length: 86, dtype: int64

In [14]:
#The number of labels in training data is way too much beyond our expectation as averagely each genre contains only 4 songs
# We look at the first 40 genres and found that most genres contain very few songs. About 10 genres are dominating in the 
# dataset.
Train_data['top genre'].value_counts()[:40]

adult standards         80
album rock              67
dance pop               62
brill building pop      16
glam rock               16
europop                 14
dance rock              13
boy band                10
british invasion         8
bubblegum dance          7
disco                    7
art rock                 7
eurodance                6
barbadian pop            6
deep adult standards     6
atl hip hop              6
british soul             5
soft rock                5
pop                      5
classic soul             5
east coast hip hop       4
doo-wop                  4
classic uk pop           4
g funk                   3
new wave pop             3
disco house              3
hip hop                  2
bronx hip hop            2
chanson                  2
glam metal               2
alternative metal        2
celtic rock              2
blues rock               2
permanent wave           2
chicago rap              2
classic rock             2
detroit hip hop          2
c

In [15]:
# We copy our training data for the convenience of avoid confusion of variables names and we can always later on retrieve 
# it. 
Train_Data = Train_data.copy()
Test_Data = Test_data.copy()

# 2. Application of basic classifiers
### We first apply some basic classification methods, especially non-ensemble methods, in order to get a broad sense of roughly how much is the accuracy on the raw dataset. We have chosen to do with a linear model and a tree model to see the difference. 
### To get started, training data and training label are prepared at this stage. We do not apply any advanced or complicated data processing or engineering at this early stage and only use the raw numerical data. Note that we also drop the 'year' as some songs have been re-released and thus the year data is not accurate and can be misleading.

In [16]:
Train_label = Train_Data['top genre']
Train_data = Train_data.drop(['Id','title','artist','year','top genre'],1)
Test_data = Test_data.drop(['Id','title','artist','year'],1)

In [17]:
Test_label = Test_data_label['top genre']

## 2.1 First Classification method applied: SVC
### Since SVC is not a tree model, we also need to scale the numerical data to avoid bias.

In [18]:
svc_clf = make_pipeline(StandardScaler(), SVC(kernel='rbf', C = 2,random_state = 20))

In [19]:
svc_clf.fit(Train_data,Train_label)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=2, random_state=20))])

In [20]:
pred = svc_clf.predict(Test_data)

In [21]:
accuracy_score(Test_label,pred)

0.34513274336283184

### After some tuning and testing, we found that SVC could give us an accuracy around 35% at most without any data processing or feature engineering. Next we are going to look at how we can acquire some useful data by feature engineering.

### 2.1.1 Trial of Feature Engineering
### We observe that 'bpm' stands for beats per minute (tempo) and 'dur' for duration: the length of the song. We wonder it would be useful that we create a new numerical feature by multiplying these 2 columns to stand for total beats?

In [22]:
Train_data['total beats'] = Train_data['bpm']*Train_data['dur']/60
Test_data['total beats'] = Test_data['bpm']*Test_data['dur']/60

In [23]:
# Now we have a new column and let's see how much accuracy it could potentially increase.
Train_data

,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,total beats
0,107,31,45,-8,13,28,150,75,3,44,267.500000
1,114,44,53,-8,13,47,139,49,3,37,264.100000
2,105,36,63,-9,13,67,245,11,3,77,428.750000
3,170,28,47,-16,13,33,232,25,3,67,657.333333
4,121,47,56,-8,15,40,193,45,3,63,389.216667
...,...,...,...,...,...,...,...,...,...,...,...
448,80,22,18,-17,10,16,214,92,4,45,285.333333
449,148,81,53,-13,23,96,147,50,3,50,362.600000
450,168,55,73,-8,20,61,289,23,14,77,809.200000
451,165,87,64,-5,6,88,191,5,8,62,525.250000


In [24]:
svc_clf = make_pipeline(StandardScaler(), SVC(kernel='rbf', C = 2,random_state=20))

In [25]:
svc_clf.fit(Train_data,Train_label)
pred = svc_clf.predict(Test_data)

In [26]:
accuracy_score(Test_label,pred)

0.35398230088495575

### The accuracy barely improves a little! Our first trial of data engineering has succeeded.

## 2.2  Second Classification method applied: DecisionTree

### 2.2.1 We apply DecisionTree on both dataset again, just to test how is the effect of previous feature engineering 

In [27]:
# Refresh data
Train_data = Train_Data.drop(['Id','title','artist','year','top genre'],1)
Test_data = Test_Data.drop(['Id','title','artist','year'],1)

In [28]:
DT_clf = DecisionTreeClassifier(max_depth = 3,min_samples_split=3)

In [29]:
np.mean(cross_val_score(DT_clf, Train_data, Train_label, cv=20))

0.28320158102766796

### The accuracy from DecisionTree is about 29% at most, a considerable 6 % short than SVC. 

### 2.2.2  Apply DecisionTree on engineered dataset

In [30]:
Train_data['total beats'] = Train_data['bpm']*Train_data['dur']/60
Test_data['total beats'] = Test_data['bpm']*Test_data['dur']/60

In [31]:
DT_clf = DecisionTreeClassifier(max_depth = 3,min_samples_split=3)

In [32]:
np.mean(cross_val_score(DT_clf, Train_data, Train_label, cv=20))

0.28320158102766796

## 2.3. Conclusion from naive application of basic tree and linear models on raw data, with effectless data engineering conducted. 

### From the above results, we can see that the basic classifiers could give us an accuracy of upper bound at 35% at best, with SVC applied, and lower bound of 29% with DecisionTree. We have conducted some basic feature engineering but the effect is not significant. We will endeavour to put more effort on data engineering and data processing in the following part.

# 3. Dive into data engineering and data processing.

## 3.1 Some observations before data engineering

### Recall from previous section that we simply dropped the categorical features and text features for simplicity. Now let's examine them again to see what's useful and how to make use of them. 

### Obviously the artists' names are quite useful cause they indicate where they from and their other information could be acquired on the Internet. We also notice that some genres contain nationality, which hints that by adding artists' nationalities to the dataset, we might stand a higher chance of getting a higher accuracy!

## 3.2 Getting artists' nationalities using a package called Name2nat

In [33]:
# Instantiate the class
my_name2nat = Name2nat()

2022-02-21 10:43:27,714 loading file D:\anacon\Anaconda\Lib\site-packages\name2nat/best-model.pt


In [34]:
# To get the nationalities for test data as well, we combine training data and test data at this point.
combine = pd.concat([Train_Data,Test_Data]) # Do not forget to obtain the raw and intact data.

In [35]:
name_list = combine['artist'].to_list() # Turning 'artist' into a list  
nation_list = []                        # Set an empty list to collect nationalities.
for name in name_list:
    
    result = my_name2nat(name)
    
    if not result[0][1]:
        
        nation_list.append('')
    
    else :
        nation_list.append(result[0][1][0][0])

In [36]:
print(nation_list)     # Have a look at the result

['American', '', '', 'American', 'English', 'American', '', '', 'English', 'American', '', 'English', '', 'English', 'American', '', 'American', '', '', 'American', '', 'American', '', '', 'American', '', 'American', 'American', '', 'American', 'American', 'American', 'Barbadian', 'American', 'American', 'American', '', 'American', '', 'American', '', 'American', '', '', '', 'American', 'American', '', '', '', '', '', '', '', 'English', '', 'English', 'American', 'American', '', '', '', '', 'English', 'American', 'American', 'American', 'American', '', 'American', '', '', 'American', 'Jamaican', '', '', '', 'English', 'American', 'British', '', 'English', 'American', '', 'English', 'American', '', 'American', '', 'American', 'American', 'English', '', '', '', 'American', '', '', 'English', '', 'American', '', '', '', '', '', 'American', 'Belgian', '', 'British', '', 'German', 'American', 'Italian', 'American', '', '', 'American', 'English', '', 'American', 'American', 'English', '', 'A

### We can see that some values are missing. That's because the package isn't that powerful so as to obtain the complete information. We will bear with it at the moment and test how much improvement is this information bringing.

In [37]:
combine['Nationality'] = nation_list
combine

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top genre,Nationality
0,1,My Happiness,Connie Francis,1996,107,31,45,-8,13,28,150,75,3,44,adult standards,American
1,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,47,139,49,3,37,adult standards,
2,3,How Deep Is Your Love,Bee Gees,1979,105,36,63,-9,13,67,245,11,3,77,adult standards,
3,4,Woman in Love,Barbra Streisand,1980,170,28,47,-16,13,33,232,25,3,67,adult standards,American
4,5,Goodbye Yellow Brick Road - Remastered 2014,Elton John,1973,121,47,56,-8,15,40,193,45,3,63,glam rock,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,563,Candy Shop,50 Cent,2005,125,57,61,-8,38,76,209,3,47,78,NaN,American
109,564,Dragostea Din Tei - Italian Version,O-Zone,2010,130,89,67,-6,10,80,215,4,3,44,NaN,
110,565,Big Poppa - 2005 Remaster,The Notorious B.I.G.,1994,84,58,78,-7,14,76,253,43,27,74,NaN,American
111,566,YMCA - Original Version 1978,Village People,1978,127,97,72,-5,12,73,287,6,14,71,NaN,


In [38]:
combine = combine.drop(['Id','year','artist','title','top genre'],1)

In [39]:
Train_data_dummy = pd.get_dummies(combine)
Train_data_dummy

,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,...,Nationality_Belgian,Nationality_British,Nationality_Canadian,Nationality_English,Nationality_French,Nationality_German,Nationality_Italian,Nationality_Jamaican,Nationality_Mexican,Nationality_Turk
0,107,31,45,-8,13,28,150,75,3,44,...,0,0,0,0,0,0,0,0,0,0
1,114,44,53,-8,13,47,139,49,3,37,...,0,0,0,0,0,0,0,0,0,0
2,105,36,63,-9,13,67,245,11,3,77,...,0,0,0,0,0,0,0,0,0,0
3,170,28,47,-16,13,33,232,25,3,67,...,0,0,0,0,0,0,0,0,0,0
4,121,47,56,-8,15,40,193,45,3,63,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,125,57,61,-8,38,76,209,3,47,78,...,0,0,0,0,0,0,0,0,0,0
109,130,89,67,-6,10,80,215,4,3,44,...,0,0,0,0,0,0,0,0,0,0
110,84,58,78,-7,14,76,253,43,27,74,...,0,0,0,0,0,0,0,0,0,0
111,127,97,72,-5,12,73,287,6,14,71,...,0,0,0,0,0,0,0,0,0,0


In [40]:
Train_data_dummy.columns 

Index(['bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch',
       'pop', 'Nationality_', 'Nationality_American', 'Nationality_Angolan',
       'Nationality_Australian', 'Nationality_Barbadian',
       'Nationality_Belgian', 'Nationality_British', 'Nationality_Canadian',
       'Nationality_English', 'Nationality_French', 'Nationality_German',
       'Nationality_Italian', 'Nationality_Jamaican', 'Nationality_Mexican',
       'Nationality_Turk'],
      dtype='object')

In [41]:
Train_data_dummy = Train_data_dummy.drop(['Nationality_American'],1)     # Be careful of dummy variable traps

In [42]:
Train_data = Train_data_dummy[:452]      # First 452 row are from training set
Test_data = Train_data_dummy[452:]       

### We have our data reinforced and ready for classification now. We will use 2 ensemble methods: RandomForestClassifier and Xgboost for classification in the coming part. 

## 3.3 Third classification method applied: RandomForest

In [43]:
Random_clf = RandomForestClassifier(max_depth =5,n_estimators = 400,random_state = 20)
Random_clf.fit(Train_data,Train_label)

RandomForestClassifier(max_depth=5, n_estimators=400, random_state=20)

In [44]:
Pred = Random_clf.predict(Train_data)

accuracy_score(Train_label,Pred)

0.45132743362831856

In [45]:
Pred_test = Random_clf.predict(Test_data)

accuracy_score(Test_label,Pred_test)

0.36283185840707965

### We received 36.3%, a slight increase of 1% is kind of boosting our confidence, as this accuracy has never been reached before! At this stage we believe our data engineering is at least useful to some extent.

## 3.4 Fourth classification method: Xgboost

### Xgboost is a very recent model created by Tianqi Chen, who is a very talented AI researcher. He developed this model during his Phd time in New York University. This model has performed very well in all kinds of Kaggle competitions. Let's try it out!.

In [46]:
xgb_cl = xgb.XGBClassifier(n_estimators=100, 
                           subsample=0.6, 
                           max_depth=2, 
                           learning_rate=0.1, 
                           colsample_bytree=0.4,
                            min_split_loss =0.3,
                           random_state = 20
             #               grow_policy='lossguide',
             #              max_leaves = 10
                           
                           )
xgb_cl.fit(Train_data,Train_label)

[10:43:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4,
              enable_categorical=False, gamma=0.300000012, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, min_split_loss=0.3, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=20, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=0.6, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [47]:
xgb_pred = xgb_cl.predict(Train_data)

accuracy_score(Train_label,xgb_pred)

0.6880530973451328

In [48]:
Pred_test = Random_clf.predict(Test_data)

accuracy_score(Test_label,Pred_test)

0.36283185840707965

### Not so much as we expected it would perform! It gives us the same accuracy as RandomForestClassifier does. However at least we know it is not bad. We now have explored all 4 models on the dataset, and we can see that RandomForest and Xgboost are among the best. In the following part we will focus on more data engineering and see how far we could push the accuracy.

# 4. More Data Engineering

### From the previous part, we know that data engineering is very important in increasing model performance. We continue diving deeper into this area now.

## 4.1 Examine the incompleteness of our data

In [49]:
# We can see 265 out of 565 rows are missing nationalites and we are to fill them all !
combine[combine['Nationality'] == ''] 

,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,Nationality
1,114,44,53,-8,13,47,139,49,3,37,
2,105,36,63,-9,13,67,245,11,3,77,
6,128,48,48,-14,5,12,219,8,6,44,
7,150,43,64,-9,6,53,351,7,2,71,
10,98,74,72,-3,17,93,224,12,3,63,
...,...,...,...,...,...,...,...,...,...,...,...
105,128,53,69,-11,7,57,292,11,4,66,
106,124,75,79,-11,14,71,226,27,4,67,
109,130,89,67,-6,10,80,215,4,3,44,
111,127,97,72,-5,12,73,287,6,14,71,


## 4.2 Filling nationality and sexuality information

In [50]:
# In this part we manually fill in all the desired information. The code is very clumsy so we save it as a csv file and 
# simply load it for simplicity. The code looks like this

# combine.loc[combine['artist'] == 'Queen','form'] = 'group'
# combine.loc[combine['artist'] == 'Queen','Nationality'] = 'British'

# We fill in 'form' and 'Nationality' for the artists. 'Form' has 3 options: 'male', 'female' and 'group.'

## 4.3 More detailed information to complement

In [51]:
# Again we fill in more detailed information

# combine.loc[combine['artist'] == 'Elton John', ['male','female','white','black','active']] = [1,0,1,0,1967]

# This time the code looks like above, where 'male','female','white','black' stands for the number of that kind in the 
# artist, respectively. 'Active' stands for the year from which they became active in music career.

In [52]:
# We now load our data.
combine = pd.read_csv(r'D:\Jupyter Notebooks\CS985 Assignment1\enhanceddata.csv')

# 'white' and 'black' stand for the number of people of that race for that artist. 
# 'active' stands for the year in which the artist became active. We use it as a replacement for the inaccurate 'year'.
# 'form' was the initial idea but was replaced by assigning specific values in 'male' and 'female' columns.
combine

,Unnamed: 0,Id,title,artist,year,bpm,nrgy,dnce,dB,live,...,spch,pop,top genre,form,Nationality,male,female,white,black,active
0,0,1,My Happiness,Connie Francis,1996,107,31,45,-8,13,...,3,44,adult standards,female,NaN,0.0,1.0,1.0,0.0,1943.0
1,1,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,...,3,37,adult standards,group,American,2.0,1.0,3.0,0.0,1957.0
2,2,3,How Deep Is Your Love,Bee Gees,1979,105,36,63,-9,13,...,3,77,adult standards,group,Australian,3.0,0.0,3.0,0.0,1958.0
3,3,4,Woman in Love,Barbra Streisand,1980,170,28,47,-16,13,...,3,67,adult standards,female,NaN,0.0,1.0,1.0,0.0,1960.0
4,4,5,Goodbye Yellow Brick Road - Remastered 2014,Elton John,1973,121,47,56,-8,15,...,3,63,glam rock,male,NaN,1.0,0.0,1.0,0.0,1967.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,108,563,Candy Shop,50 Cent,2005,125,57,61,-8,38,...,47,78,NaN,male,NaN,1.0,0.0,0.0,1.0,1996.0
561,109,564,Dragostea Din Tei - Italian Version,O-Zone,2010,130,89,67,-6,10,...,3,44,NaN,group,Moldova,4.0,0.0,4.0,0.0,1999.0
562,110,565,Big Poppa - 2005 Remaster,The Notorious B.I.G.,1994,84,58,78,-7,14,...,27,74,NaN,male,NaN,1.0,0.0,0.0,1.0,1992.0
563,111,566,YMCA - Original Version 1978,Village People,1978,127,97,72,-5,12,...,14,71,NaN,group,American,6.0,0.0,3.0,3.0,1977.0


In [53]:
combine_copy = combine.drop(['Id','year','artist','title','top genre','form'],1)   # Backup

In [54]:
Train_data_dummy = pd.get_dummies(combine_copy)                     # Get dummy variables
Train_data_dummy = Train_data_dummy.drop(['Nationality_American'],1)     # Avoid dummy variable trap

In [55]:
# Split the data into training and test sets.
Train_data = Train_data_dummy[:452]                 
Test_data = Train_data_dummy[452:]

## 4.4 Trying our data on RandomForest and Xgboost

In [56]:
Random_clf = RandomForestClassifier(max_depth =9,n_estimators = 160,random_state =20)
Random_clf.fit(Train_data,Train_label)

RandomForestClassifier(max_depth=9, n_estimators=160, random_state=20)

In [57]:
Pred = Random_clf.predict(Train_data)

accuracy_score(Train_label,Pred)

0.915929203539823

In [58]:
Pred_test = Random_clf.predict(Test_data)

accuracy_score(Test_label,Pred_test)

0.4424778761061947

### We have received an accuracy of 46%! Significantly higher than previously! This tells us that Data Engineering is really the most important part in machine learning as 'rubbish in rubbish out' tells us the same. Next we try Xgboost and compare their performances.

In [59]:
xgb_cl = xgb.XGBClassifier(n_estimators=120, 
                           subsample=0.7, 
                           max_depth=2, 
                           learning_rate=0.12, 
                           colsample_bytree=0.5,
                            min_split_loss =0.35,
                           random_state = 22
             #               grow_policy='lossguide',
             #              max_leaves = 10
                           
                           )
xgb_cl.fit(Train_data,Train_label)

[10:43:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, gamma=0.349999994, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.12, max_delta_step=0, max_depth=2,
              min_child_weight=1, min_split_loss=0.35, missing=nan,
              monotone_constraints='()', n_estimators=120, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=22, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=0.7, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [60]:
xgb_pred = xgb_cl.predict(Train_data)

accuracy_score(Train_label,xgb_pred)

0.8539823008849557

In [61]:
xgb_test_pred = xgb_cl.predict(Test_data)

accuracy_score(Test_label,xgb_test_pred)

0.4690265486725664

### Xgboost is now doing better than RandomForest by about 3%. That's a considerable difference in machine learning! We will go along with Xgboost in the following section for this reason.
### Recall we used bmp*dur previouly and it did not work very well. Let's try it again at this stage.

## 4.6 Generating new features from existing features.

In [62]:
combine_copy = Train_data_dummy.copy()            # Save Train_data_dummy as a safe copy 

combine['bpm*dur'] = combine['bpm'] *combine['dur']/60

Train_data = combine_copy[:452]                 
Test_data = combine_copy[452:]

In [63]:
xgb_cl = xgb.XGBClassifier(n_estimators=140, 
                           subsample=0.7, 
                           max_depth=5, 
                           learning_rate=0.12, 
                           colsample_bytree=0.5,
                            min_split_loss =0.35,
                           random_state = 27
             #               grow_policy='lossguide',
             #              max_leaves = 10
                           
                           )
xgb_cl.fit(Train_data,Train_label)

[10:43:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, gamma=0.349999994, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.12, max_delta_step=0, max_depth=5,
              min_child_weight=1, min_split_loss=0.35, missing=nan,
              monotone_constraints='()', n_estimators=140, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=27, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=0.7, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [64]:
xgb_pred = xgb_cl.predict(Train_data)

accuracy_score(Train_label,xgb_pred)

0.9070796460176991

In [65]:
xgb_test_pred = xgb_cl.predict(Test_data)

accuracy_score(Test_label,xgb_test_pred)

0.4778761061946903

### Seems Feature Engineering is of some help. The accuracy increases by 1%, although it could just be lucky. Let's perform some more powerful operations on our dataset and see what will happen.

In [66]:
# Refresh dataset
combine_copy = Train_data_dummy.copy()

In [67]:
# Save those dummy varibles. We do not want to generate new numerical features with them.
save = combine_copy.iloc[:,10:]

combine_copy.drop(combine_copy.iloc[:,10:], inplace = True, axis = 1)

In [68]:
# Feature tools
es = ft.EntitySet(id='single_dataframe')
es = es.add_dataframe(
    dataframe_name="combined",
    dataframe=combine_copy,
    index="index"
    )

In [69]:
# To not increase feature dimensions incredibly to a huge amount, we only use multiplication to generate new features.
trans_primitives=['multiply_numeric']

feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_dataframe_name="combined",
                                      trans_primitives=trans_primitives,
                                      max_depth=1
                                     )
                                                 

In [70]:
feature_matrix

,Unnamed: 0,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,...,dur * val,live * Unnamed: 0,live * nrgy,live * spch,live * val,nrgy * Unnamed: 0,nrgy * spch,nrgy * val,spch * Unnamed: 0,spch * val
index,,,,,,,,,,,,,,,,,,,,,
0,0,107,31,45,-8,13,28,150,75,3,...,4200.0,0.0,403.0,39.0,364.0,0.0,93.0,868.0,0.0,84.0
1,1,114,44,53,-8,13,47,139,49,3,...,6533.0,13.0,572.0,39.0,611.0,44.0,132.0,2068.0,3.0,141.0
2,2,105,36,63,-9,13,67,245,11,3,...,16415.0,26.0,468.0,39.0,871.0,72.0,108.0,2412.0,6.0,201.0
3,3,170,28,47,-16,13,33,232,25,3,...,7656.0,39.0,364.0,39.0,429.0,84.0,84.0,924.0,9.0,99.0
4,4,121,47,56,-8,15,40,193,45,3,...,7720.0,60.0,705.0,45.0,600.0,188.0,141.0,1880.0,12.0,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,108,125,57,61,-8,38,76,209,3,47,...,15884.0,4104.0,2166.0,1786.0,2888.0,6156.0,2679.0,4332.0,5076.0,3572.0
561,109,130,89,67,-6,10,80,215,4,3,...,17200.0,1090.0,890.0,30.0,800.0,9701.0,267.0,7120.0,327.0,240.0
562,110,84,58,78,-7,14,76,253,43,27,...,19228.0,1540.0,812.0,378.0,1064.0,6380.0,1566.0,4408.0,2970.0,2052.0


In [71]:
save = save.reset_index()

In [72]:
combine_copy = pd.concat([feature_matrix,save],1)

In [73]:
Train_data = combine_copy[:452]                 
Test_data = combine_copy[452:]

In [74]:
xgb_cl = xgb.XGBClassifier(n_estimators=100, 
                           subsample=0.7, 
                           max_depth=5, 
                           learning_rate=0.05, 
                           colsample_bytree=0.5,
                            min_split_loss =0.2,
                           random_state = 24
             #               grow_policy='lossguide',
             #              max_leaves = 10
                           
                           )
xgb_cl.fit(Train_data,Train_label)

[10:43:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, gamma=0.200000003, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=1, min_split_loss=0.2, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=24, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=0.7, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [75]:
xgb_pred = xgb_cl.predict(Train_data)

accuracy_score(Train_label,xgb_pred)

0.827433628318584

In [76]:
xgb_test_pred = xgb_cl.predict(Test_data)

accuracy_score(Test_label,xgb_test_pred)

0.40707964601769914

### Looks like Featuretool isn't doing well. At this stage, we feel that data obtained from real world is more realiable and informative than these artifical ones, who only destroy the original data structure and statistics relationship. I have also tried all kinds of SMOTE techniques, which is used for resampling on imbalanced dataset like the one we are working on, but their performances are rather poor as well. For simplicity, I just avoid putting the results from SMOTE in this notebook.
### Now we know that by manually adding data obtained from real world, it is very promising to increase  machine learning model's performance. However, it is very labour consuming as you have seen.
### The model we use is also quite important. As in our case, the SVC performs better than DecisionTree, but it is outperformed by the ensemble models, which are more powerful generally. Xgboost performs particularly well among ensemble models.
### Last thing to note: Tuning parameters is also very crucial in the sucess of building an accurate model, for us Xgboost in particular. It could perform very poorly if not well set, but also performs the best when the parameters are the right ones.
### We finish our report by apply sentiment analysis to the title, which we almost forget to use!

## 4.6 Sentiment analysis on title

In [77]:
# Refresh data
combine_copy = combine.copy()

In [78]:
combine_copy

,Unnamed: 0,Id,title,artist,year,bpm,nrgy,dnce,dB,live,...,pop,top genre,form,Nationality,male,female,white,black,active,bpm*dur
0,0,1,My Happiness,Connie Francis,1996,107,31,45,-8,13,...,44,adult standards,female,NaN,0.0,1.0,1.0,0.0,1943.0,267.500000
1,1,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,...,37,adult standards,group,American,2.0,1.0,3.0,0.0,1957.0,264.100000
2,2,3,How Deep Is Your Love,Bee Gees,1979,105,36,63,-9,13,...,77,adult standards,group,Australian,3.0,0.0,3.0,0.0,1958.0,428.750000
3,3,4,Woman in Love,Barbra Streisand,1980,170,28,47,-16,13,...,67,adult standards,female,NaN,0.0,1.0,1.0,0.0,1960.0,657.333333
4,4,5,Goodbye Yellow Brick Road - Remastered 2014,Elton John,1973,121,47,56,-8,15,...,63,glam rock,male,NaN,1.0,0.0,1.0,0.0,1967.0,389.216667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,108,563,Candy Shop,50 Cent,2005,125,57,61,-8,38,...,78,NaN,male,NaN,1.0,0.0,0.0,1.0,1996.0,435.416667
561,109,564,Dragostea Din Tei - Italian Version,O-Zone,2010,130,89,67,-6,10,...,44,NaN,group,Moldova,4.0,0.0,4.0,0.0,1999.0,465.833333
562,110,565,Big Poppa - 2005 Remaster,The Notorious B.I.G.,1994,84,58,78,-7,14,...,74,NaN,male,NaN,1.0,0.0,0.0,1.0,1992.0,354.200000
563,111,566,YMCA - Original Version 1978,Village People,1978,127,97,72,-5,12,...,71,NaN,group,American,6.0,0.0,3.0,3.0,1977.0,607.483333


In [79]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')

2022-02-21 10:43:42 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

2022-02-21 10:43:42 INFO: Use device: cpu
2022-02-21 10:43:42 INFO: Loading: tokenize
2022-02-21 10:43:42 INFO: Loading: sentiment
2022-02-21 10:43:43 INFO: Done loading processors!


In [80]:
title = combine_copy['title']
title = list(title)

In [81]:
emotion = []

for i in range(len(title)):
    doc = nlp(title[i])
    for i, sentence in enumerate(doc.sentences):
        emotion.append(sentence.sentiment)
        break

In [82]:
# We have 3 types of sentiment here. 0 for negative, 1 for neutral and 2 for positive
senti = []                             
for i in range(len(emotion)):
    
    if emotion[i] ==0:
        senti.append('negative')
        
    elif emotion[i] ==1:
        senti.append('neutral')
        
    else:
        senti.append('positive')

In [83]:
combine_copy['sentiment'] = senti

In [84]:
combine_copy

,Unnamed: 0,Id,title,artist,year,bpm,nrgy,dnce,dB,live,...,top genre,form,Nationality,male,female,white,black,active,bpm*dur,sentiment
0,0,1,My Happiness,Connie Francis,1996,107,31,45,-8,13,...,adult standards,female,NaN,0.0,1.0,1.0,0.0,1943.0,267.500000,neutral
1,1,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,...,adult standards,group,American,2.0,1.0,3.0,0.0,1957.0,264.100000,neutral
2,2,3,How Deep Is Your Love,Bee Gees,1979,105,36,63,-9,13,...,adult standards,group,Australian,3.0,0.0,3.0,0.0,1958.0,428.750000,positive
3,3,4,Woman in Love,Barbra Streisand,1980,170,28,47,-16,13,...,adult standards,female,NaN,0.0,1.0,1.0,0.0,1960.0,657.333333,neutral
4,4,5,Goodbye Yellow Brick Road - Remastered 2014,Elton John,1973,121,47,56,-8,15,...,glam rock,male,NaN,1.0,0.0,1.0,0.0,1967.0,389.216667,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,108,563,Candy Shop,50 Cent,2005,125,57,61,-8,38,...,NaN,male,NaN,1.0,0.0,0.0,1.0,1996.0,435.416667,neutral
561,109,564,Dragostea Din Tei - Italian Version,O-Zone,2010,130,89,67,-6,10,...,NaN,group,Moldova,4.0,0.0,4.0,0.0,1999.0,465.833333,neutral
562,110,565,Big Poppa - 2005 Remaster,The Notorious B.I.G.,1994,84,58,78,-7,14,...,NaN,male,NaN,1.0,0.0,0.0,1.0,1992.0,354.200000,neutral
563,111,566,YMCA - Original Version 1978,Village People,1978,127,97,72,-5,12,...,NaN,group,American,6.0,0.0,3.0,3.0,1977.0,607.483333,neutral


In [85]:
combine_copy = combine_copy.drop(['Id','title','artist','year','top genre','form'],1)

In [86]:
combine_dummy = pd.get_dummies(combine_copy)                     # Get dummy variables
combine_dummy = combine_dummy.drop(['sentiment_negative','Nationality_American'],1)

In [87]:
Train_data = combine_dummy[:452]                 
Test_data = combine_dummy[452:]

In [88]:
xgb_cl = xgb.XGBClassifier(n_estimators=90, 
                           subsample=0.7, 
                           max_depth=2, 
                           learning_rate=0.14, 
                           colsample_bytree=0.5,
                            min_split_loss =0.2,
                           random_state = 24
             #               grow_policy='lossguide',
             #              max_leaves = 10
                           
                           )
xgb_cl.fit(Train_data,Train_label)

[10:43:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, gamma=0.200000003, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.14, max_delta_step=0, max_depth=2,
              min_child_weight=1, min_split_loss=0.2, missing=nan,
              monotone_constraints='()', n_estimators=90, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=24, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=0.7, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [89]:
xgb_pred = xgb_cl.predict(Train_data)

accuracy_score(Train_label,xgb_pred)

0.8517699115044248

In [90]:
xgb_test_pred = xgb_cl.predict(Test_data)

accuracy_score(Test_label,xgb_test_pred)

0.4778761061946903

# 5. Some conclusions 
### The title's sentiment doesn't seem to make a considerable impact. We are now very clear that not all data is useful. It really depends on the task. Simply piling data won't give people a satisfactory result. The data scientists need to think carefully and obtain data specific for the mission so that good outcome would be expected. 